# Generating meaningless images that are high in "meaningful information"
**by Tom English**

In [1]:
!TZ=America/Chicago date

Sun Oct 27 01:56:05 CDT 2019


For orientation, see my post "<a href="http://theskepticalzone.com/wp/evo-info-5-meaningless-meaning/">Evo-Info 5: Meaningless Meaning</a>" at <em>The Skeptical Zone.</em> 

In the following, I use small amounts of code to transform large images in the Wikimedia Commons into large, meaningless images that are compressed little when stored in PNG format. Then I explain how the generated images serve as counterexamples to the claim of intelligent-design proponents that algorithmic specified complexity is a measure of meaningful information.

## Generation of images

First load some image-processing functions.

In [2]:
# The code is included in the binder containing this notebook.
%run image_processing.py

### Image processing

Specify the URLs of images of Mount Fuji and Mount Rushmore in the Wikimedia Commons.

In [3]:
# Note to programmers: The paths to images do not have to be URLs. You
# can replace them with paths to local files. The images can be stored
# in any of the common formats. However, the images must be identical in
# depth.

fuji_path = ("https://upload.wikimedia.org/wikipedia/commons/"
              + "f/f7/Mount_Fuji_Japan_with_Snow%2C_Lakes_and_"
              + "Surrounding_Mountains.jpg")
rush_path = ("https://upload.wikimedia.org/wikipedia/commons/"
              + "5/58/Aerial_view_of_Mount_Rushmore_National_Memorial_"
              + "by_Volkan_Yuksel_DSC04244.JPG")

Read the Fuji and Rushmore images, and crop them to the regions that
overlap when they are aligned at their upper-left corners. Then report
the dimensions and sizes of the images.

In [4]:
fuji, rush = overlap(imread(fuji_path), imread(rush_path))
n_raw_bits = 8 * np.prod(fuji.shape)
print('Image height, width, and depth :', fuji.shape)
print('Number of bits per raw image   : {:,}'.format(n_raw_bits))

Image height, width, and depth : (2592, 3456, 3)
Number of bits per raw image   : 214,990,848


Meld the images of Fuji and Rushmore into a new image. For each pair of corresponding 8-bit RGB values in the two images:

1. Calculate the [bitwise OR](https://en.wikipedia.org/wiki/Bitwise_operation#OR) and the [bitwise AND](https://en.wikipedia.org/wiki/Bitwise_operation#AND) of the RGB values.
2. Calculate the 8-bit product of the OR result and the AND result, discarding [overflow](https://en.wikipedia.org/wiki/Integer_overflow).
3. Calculate the [bitwise NOT](https://en.wikipedia.org/wiki/Bitwise_operation#NOT) of the product.


You can obtain elementary explanations of the operations by clicking on the links I have provided. Whether or not you understand the details, the main points to grasp are that 

<ol style="list-style-type: upper-alpha;">
    <li>the meld requires very little code,</li>
    <li>there is loss of information in the sense that the transformation is irreversible, and</li>
    <li>the resulting image is identical in dimensions to the images of Fuji and Rushmore.</li>
</ol>

In [5]:
meld = ~((fuji | rush) * (fuji & rush))

Create an image that is the cumulative 8-bit sum (with overflow bits
discarded) of RGB values in the Rushmore image. For a simple description of this operation, along with an explanation of why it entails loss of information, see "[Evo-Info 4: Non-conservation of algorithmic specified complexity](http://theskepticalzone.com/wp/evo-info-4-non-conservation-of-algorithmic-specified-complexity/)."

In [6]:
csum = cumulative_sum(rush)

Write all of the images to disk in Portable Network Graphics (PNG) format, causing the images to be compressed without loss (meaning that decompression of a compressed image gives precisely the image that was compressed). Report the sizes of the compressed images. The name of the file containing an image is the title of the image, extended with '.png'.

In [7]:
print('Number of bits in each of the compressed images')
formatter = '   {:26} : {:11,}'
n_fuji = write_png(fuji, 'Fuji at Dawn', formatter)
n_rush = write_png(rush, 'ET Views Rushmore', formatter)
n_meld = write_png(meld, 'The Languor of Egnorance', formatter)
n_csum = write_png(csum, 'The Apotheosis of Meaning', formatter)

Number of bits in each of the compressed images
   Fuji at Dawn               :  69,143,232
   ET Views Rushmore          : 102,385,888
   The Languor of Egnorance   : 193,402,368
   The Apotheosis of Meaning  : 206,639,528


### Display of images

The following images are scaled down to fit within the cell. To view an image at full resolution (at least in some browsers), click on its title. Note to programmers: After running the notebook, you must manually update this Markdown cell to refresh the images.

[*Fuji at Dawn*](./Fuji%20at%20Dawn.png).  Photo credit: Kpravin2 via <a href="https://commons.wikimedia.org/wiki/File:Mount_Fuji_Japan_with_Snow,_Lakes_and_Surrounding_Mountains.jpg">Wikimedia Commons</a> (<a href="https://creativecommons.org/licenses/by-sa/4.0/deed.en">CC BY-SA 4.0</a>)
<img src="./Fuji%20at%20Dawn.png">

&nbsp;

[*E.T. Views Rushmore*](./ET%20Views%20Rushmore.png). Photo credit: Volkan Yuksel, <a href="https://commons.wikimedia.org/wiki/File:Aerial_view_of_Mount_Rushmore_National_Memorial_by_Volkan_Yuksel_DSC04244.JPG">via Wikimedia Commons</a> (<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC BY-SA 3.0</a>).
<img src="./ET%20Views%20Rushmore.png">

&nbsp;

[*The Languor of Egnorance*](./The%20Languor%20of%20Egnorance.png)
<img src="./The%20Languor%20of%20Egnorance.png">

&nbsp;

[*The Apotheosis of Meaning*](./The%20Apotheosis%20of%20Meaning.png)
<img src="./The%20Apotheosis%20of%20Meaning.png">

## Discussion

Intelligent-design proponents Winston Ewert, William Dembski, and Robert Marks regard *algorithmic specified complexity* (ASC) as a measure of meaningful information. We shall follow their article "<a href="http://robertmarks.org/REPRINTS/2015%20Measuring%20meaningful%20information%20in%20images.pdf">Measuring Meaningful Information in Images: Algorithmic Specified Complexity</a>" (2015) in approximating the ASC of an image:

    ASC(image) = size(compressed(image)) - K(image|context) bits,

where `size(compressed(image))` is the number of bits in the image when it is losslessly compressed for storage in Portable Network Graphics (PNG) format, and `K(image|context)` is, loosely, the length of the shortest binary program that outputs `image` on input of the binary string `context`. (Note that the size of the compressed image varies somewhat from one implementation of the PNG compressor to another.) The value of `K(image|context)` indicates the size of `image` when compressed in an alternative scheme. To put the alternative scheme concretely, if Alice and Bob have agreed upon a ([Turing-complete](https://en.wikipedia.org/wiki/Turing_completeness), [prefix-free](https://en.wikipedia.org/wiki/Prefix_code), binary) programming language and the context, then Alice can transmit to Bob a short program that outputs a large image on input of the context, and Bob can run the program with input of the context to obtain the image. That is, the binary program serves as the compressed image. Thus the ASC of an image is the difference in size when it is compressed losslessly by two different approaches, one of which cannot exploit the context, and the other of which can. For rigorous mathematical analysis of ASC, see "[Evo-Info 4: Non-conservation of algorithmic specified complexity](http://theskepticalzone.com/wp/evo-info-4-non-conservation-of-algorithmic-specified-complexity/)."

The context in our approximations of ASC is the Wikimedia Commons. For each of the images above, the number of bits of code required to generate the image on input of the context&nbsp;&mdash; an upper bound on `K(image|context)`&nbsp;&mdash; is small in comparison to the number of bits in the PNG-compressed image. Thus

    ASC(image) ≈ size(compressed(image))

for each of the images. In my [article](http://theskepticalzone.com/wp/evo-info-5-meaningless-meaning/) at *The Skeptical Zone*, the reported quantity of "meaningful information" in an image is its PNG file size rounded down to the nearest integer multiple of $10^6$ bits. 

A key observation is that the "meaningful information" of *Languor* is much greater than the sum total "meaningful information" of the images that entered into its construction:

In [8]:
size_sum = n_fuji + n_rush
print('ASC of The Languor of Egnorance    :', n_meld)
print('Sum total ASC of Fuji and Rushmore :', size_sum)
print('Difference                         : ', n_meld - size_sum)

ASC of The Languor of Egnorance    : 193402368
Sum total ASC of Fuji and Rushmore : 171529120
Difference                         :  21873248


Thus we have "created" almost 22 megabits of "meaningful information" by running a short program (much less than 22 megabits in length) that melds the given images, *Fuji* and *Rushmore*, into a new image&nbsp;&mdash; no matter that there is loss of information in the sense that neither of the input images can be recovered from the new image. The absurdity is even more pronounced in the case of *Apotheosis*, in which a very short program operates only on the context image *Rushmore*. The "meaningful information" of the output of the program is about twice as great as that of *Rushmore*:

In [9]:
print('ASC of The Apotheosis of Meaning :', n_csum)
print('ASC of Rushmore                  :', n_rush)
print('Difference                       :', n_csum - n_rush)

ASC of The Apotheosis of Meaning : 206639528
ASC of Rushmore                  : 102385888
Difference                       : 104253640


That is, according to Ewert, Dembski, and Marks, there is not loss of information, but instead a gain of over 100 million bits of meaningful information, when *Rushmore* is scrambled irreversibly to produce the patently meaningless *Apotheosis of Meaning*.